In [1]:
import re
import csv
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# path
path_data_dir = os.path.dirname(os.path.dirname(os.getcwd()))

path_negative_data = path_data_dir + r'\row_data\non-thermophilic.txt'
path_positive_data = path_data_dir +  r'\row_data\thermophilic.txt'

train_data_path_part1 = path_data_dir + r'\new_data\new_features\feature_select_'
train_data_path_part2 = '_'
train_data_path_part3 = r'_gap.csv'

In [3]:
# 将正类和负类数据抽取成list
def extract_data(path):
    lines_list = []
    lines = ''
    with open(path, 'r') as f:
        for line in f.readlines():
            if '>' not in line:
                line = line.replace('\n','').replace('\t','').replace('\r','').strip()
                lines = lines + line
            else:
                lines_list.append(lines)
                lines = ''
    lines_list.remove('')
    lines_list.append(lines)
    return lines_list

In [4]:
# 标准氨基酸 ACDEFGHIKLMNPQRSTVWY
# 当特征为连续k个时，创建k的n次方个特征返回
def create_features(new_features,k):
    row_features = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
    length = len(new_features)
    features = []
    for i in range(length):
        for j in row_features:
            features.append(new_features[i] + j)
    if k > 1:
        k -= 1
        features = create_features(features,k)
    return features

In [5]:
def get_ratio(feature, data):
    # 获取某一个特征在某一类（正类或负类）中的比率
    length = len(data)
    freq = 0
    for d in data:
        if feature in d:
            freq += 1
    return freq/length

In [6]:
def get_class_ratio(features, data):
    # 获取一类在所有样本中的比率
    features_ratio = {}
    for feat in features:
        features_ratio[feat] = get_ratio(feat, data)
    return features_ratio

In [7]:
# 抽取正负样本数据
positive_data = extract_data(path_positive_data)
negative_data = extract_data(path_negative_data)

In [8]:
def get_features(psoitive_output):
    temp = []
    for i in psoitive_output.columns:
        if psoitive_output.loc['psoitive_ratio',i] <0.001:
            temp.append(i)
    return temp

In [9]:
# 创建第一个节点的未挑选特征
features =['']
for i in range(1,4):
    features = create_features(features, 1)
    psoitive_ratio = get_class_ratio(features, positive_data)
    psoitive_output = pd.DataFrame([psoitive_ratio],index = ['psoitive_ratio'])
    new_feature = get_features(psoitive_output)
    psoitive_output.drop(new_feature, axis = 1, inplace = True)
    psoitive_output = psoitive_output.sort_values('psoitive_ratio',axis = 1,ascending=False)
    
    negative_ratio = get_class_ratio(psoitive_output.columns, negative_data)
    negative_output = pd.DataFrame([negative_ratio],index = ['negative_ratio'])
    output = pd.concat([psoitive_output,negative_output],axis = 0)
    print(output)
    output = output.sort_values('negative_ratio',axis = 1)
    train_data_path_part = train_data_path_part1 + str(i) + train_data_path_part2 + str(i)+ train_data_path_part3
    if len(output.columns) > 100:
        output = output.iloc[:,:100]
        print(output)
        output.to_csv(train_data_path_part, index = None)
        features = output.columns
    else:
        print(output)
        output.to_csv(train_data_path_part, index = None)


                       A         C         D         E         F         G  \
psoitive_ratio  0.997814  0.720219  0.992350  0.997814  0.980328  0.998907   
negative_ratio  1.000000  0.813367  0.991173  0.991173  0.984868  0.997478   

                       H         I         K         L         M         N  \
psoitive_ratio  0.928962  0.995628  0.998907  0.994536  1.000000  0.977049   
negative_ratio  0.948298  0.991173  0.992434  0.998739  0.996217  0.984868   

                       P         Q         R         S         T         V  \
psoitive_ratio  0.993443  0.937705  0.997814  0.990164  0.995628  0.997814   
negative_ratio  0.982346  0.991173  0.996217  1.000000  0.996217  0.996217   

                       W         Y  
psoitive_ratio  0.781421  0.983607  
negative_ratio  0.860025  0.964691  
                       C         W         H         Y         P         F  \
psoitive_ratio  0.720219  0.781421  0.928962  0.983607  0.993443  0.980328   
negative_ratio  0.813367  0.